In [1]:
import pandas as pd

import biogeme.biogeme_logging as blog
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, bioDraws, log, MonteCarlo, PanelLikelihoodTrajectory
from biogeme.parameters import Parameters
import biogeme.database as db
from biogeme.expressions import Variable

In [2]:
logger = blog.get_screen_logger(level=blog.INFO)

In [3]:
df = pd.read_table("http://transp-or.epfl.ch/data/swissmetro.dat", sep='\t')

In [4]:
(((df.PURPOSE != 1) * (df.PURPOSE != 3) + (df.CHOICE == 0)) > 0).value_counts()

False    6768
True     3960
Name: count, dtype: int64

In [5]:
for col in df.columns:
    if df[col].isna().any():
        logger.warning(f"Column {col} contains NaN values. This may affect the model estimation.")
    if (df[col] == 99999).any():
        logger.warning(f"Column {col} contains the value 99999, which may indicate missing or invalid data. Consider handling this before estimation.")

In [6]:
database = db.Database('swissmetro', df)

GROUP = Variable('GROUP')
SURVEY = Variable('SURVEY')
SP = Variable('SP')
ID = Variable('ID')
PURPOSE = Variable('PURPOSE')
FIRST = Variable('FIRST')
TICKET = Variable('TICKET')
WHO = Variable('WHO')
LUGGAGE = Variable('LUGGAGE')
AGE = Variable('AGE')
MALE = Variable('MALE')
INCOME = Variable('INCOME')
GA = Variable('GA')
ORIGIN = Variable('ORIGIN')
DEST = Variable('DEST')
TRAIN_AV = Variable('TRAIN_AV')
CAR_AV = Variable('CAR_AV')
SM_AV = Variable('SM_AV')
TRAIN_TT = Variable('TRAIN_TT')
TRAIN_CO = Variable('TRAIN_CO')
TRAIN_HE = Variable('TRAIN_HE')
SM_TT = Variable('SM_TT')
SM_CO = Variable('SM_CO')
SM_HE = Variable('SM_HE')
SM_SEATS = Variable('SM_SEATS')
CAR_TT = Variable('CAR_TT')
CAR_CO = Variable('CAR_CO')
CHOICE = Variable('CHOICE')

exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
print(f"Removing {(((df.PURPOSE != 1) * (df.PURPOSE != 3) + (df.CHOICE == 0)) > 0).sum()} rows from the database based on the exclusion criteria.")
database.remove(exclude)

SM_COST = database.define_variable('SM_COST', SM_CO * (GA == 0))
TRAIN_COST = database.define_variable('TRAIN_COST', TRAIN_CO * (GA == 0))
CAR_AV_SP = database.define_variable('CAR_AV_SP', CAR_AV * (SP != 0))
TRAIN_AV_SP = database.define_variable('TRAIN_AV_SP', TRAIN_AV * (SP != 0))
TRAIN_TT_SCALED = database.define_variable('TRAIN_TT_SCALED', TRAIN_TT / 100)
TRAIN_COST_SCALED = database.define_variable('TRAIN_COST_SCALED', TRAIN_COST / 100)
SM_TT_SCALED = database.define_variable('SM_TT_SCALED', SM_TT / 100)
SM_COST_SCALED = database.define_variable('SM_COST_SCALED', SM_COST / 100)
CAR_TT_SCALED = database.define_variable('CAR_TT_SCALED', CAR_TT / 100)
CAR_CO_SCALED = database.define_variable('CAR_CO_SCALED', CAR_CO / 100)

Removing 3960 rows from the database based on the exclusion criteria.


In [7]:
database.data.shape

(6768, 38)

In [8]:
#database.panel('ID')

In [9]:
B_COST = Beta('B_COST', 0.0, None, None, 0)
#B_COST_S = Beta('B_COST_S', 1, None, None, 0)
#B_COST_RND = B_COST + B_COST_S * bioDraws('b_cost_rnd', 'NORMAL')
B_TIME = Beta('B_TIME', 0.0, None, None, 0)
#B_TIME_S = Beta('B_TIME_S', 1, None, None, 0)
#B_TIME_RND = B_TIME + B_TIME_S * bioDraws('b_time_rnd', 'NORMAL')


ASC_CAR_mu = Beta('ASC_CAR_mu', 0, None, None, 0)
ASC_TRAIN_mu = Beta('ASC_TRAIN_mu', 0, None, None, 1)
ASC_SM_mu = Beta('ASC_SM_mu', 0, None, None, 0)

ASC_CAR_var = Beta('ASC_CAR_var', 0, None, None, 1)  #0.437869
ASC_TRAIN_var = Beta('ASC_TRAIN_var', 0, None, None, 1)
ASC_SM_var = Beta('ASC_SM_var', 1, None, None, 0)

ASC_CAR = ASC_CAR_mu + ASC_CAR_var * bioDraws('ASC_CAR', 'NORMAL_HALTON3')  # Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = ASC_TRAIN_mu + ASC_TRAIN_var * bioDraws('ASC_TRAIN', 'NORMAL_HALTON3')  # Beta('ASC_TRAIN', 0, None, None, 0)  #  
ASC_SM = ASC_SM_mu + ASC_SM_var * bioDraws('ASC_SM', 'NORMAL_HALTON3')  # Beta('ASC_SM', 0, None, None, 0)

V1 = ASC_TRAIN + B_TIME * TRAIN_TT_SCALED + B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + B_TIME * SM_TT_SCALED + B_COST * SM_COST_SCALED
V3 = ASC_CAR + B_TIME * CAR_TT_SCALED + B_COST * CAR_CO_SCALED

V = {1: V1, 2: V2, 3: V3}
av = {1: TRAIN_AV_SP, 2: SM_AV, 3: CAR_AV_SP}

prob = models.logit(V, av, CHOICE)
#logprob = log(MonteCarlo(PanelLikelihoodTrajectory(prob)))
logprob = log(MonteCarlo(prob))

the_biogeme = bio.BIOGEME(
    database, logprob, number_of_draws=1000, seed=999   #1223
)
the_biogeme.modelName = 'b05normal_mixture'
the_biogeme.generate_pickle = False
the_biogeme.generate_html = False

Biogeme parameters read from biogeme.toml. 


In [10]:
# asc train and sd.train 0:      init:  7140.02 bg, 6816.64 jax. final: 5254.53 bg, 5255.58 jax. results are within one std dev of biogeme, except for sd.car
# asc train, sd.train, sd.car 0: init:  6795.00 bg, 6795.09 jax. final: 5255.67 bg, 5255.58 jax. results are practically identical, well within 1 std dev.

# without constrinaing sd.car, jaxlogit says it should be zero and bg says it should be one. Let's assert it either way and see what the result is.
# note there is an old bg worksheet at https://transp-or.epfl.ch/courses/dca2011/labs/lab11/exercise-session11.pdf that looks like it suggests sd.car should be 0 within on std dev (slightly different model spec)
# asc train and sd.train 0, sd.car == 1: init: 7140.02 bg, 6816.64 jax.  final: 5266.07 bg, 5264.40 jax. parameters are different, within two or three std dev I think.
# what if we let sd.car vary? if we start at 1: -0.437869
# sd.car free, try different init vals: 0, 0.1, 1

# following is the result of bg sd.car from the first result in this cell, let's see what happens when we assert it
# asc train, sd.train, sd.car 0.437869: init: 6939.61 bg, 6795.09 jax. final: 5258.97 bg, 5255.58 jax. results within 2 std err. WHY are the initial LL vals so different?

In [11]:
# 7138.81  - bg: -6795.28  (0. for time and cost, 0 and 1 for mean and std dev, no sm asc); jaxlogit: 6985.36 if sd trian is 0, same as for biogeme. why?
# 6376.00  - bg: -6088.84  (-0.5 for time and cost, 0 and 1 for mean and std dev, no sm asc)
the_biogeme.calculate_init_likelihood()

-6794.99556643367

In [12]:
#the_biogeme.id_manager.free_betas_values, 
the_biogeme.id_manager.free_betas

ElementsTuple(expressions={'B_TIME': Beta('B_TIME', 0.0, None, None, 0), 'B_COST': Beta('B_COST', 0.0, None, None, 0), 'ASC_SM_mu': Beta('ASC_SM_mu', 0, None, None, 0), 'ASC_SM_var': Beta('ASC_SM_var', 1, None, None, 0), 'ASC_CAR_mu': Beta('ASC_CAR_mu', 0, None, None, 0)}, indices={'ASC_CAR_mu': 0, 'ASC_SM_mu': 1, 'ASC_SM_var': 2, 'B_COST': 3, 'B_TIME': 4}, names=['ASC_CAR_mu', 'ASC_SM_mu', 'ASC_SM_var', 'B_COST', 'B_TIME'])

In [ ]:
results = the_biogeme.estimate()
pandas_results = results.get_estimated_parameters()

print(results.short_summary())

As the model is rather complex, we cancel the calculation of second derivatives. If you want to control the parameters, change the name of the algorithm in the TOML file from "automatic" to "simple_bounds" 
*** Initial values of the parameters are obtained from the file __b05normal_mixture.iter 


Cannot read file __b05normal_mixture.iter. Statement is ignored. 
The number of draws (1000) is low. The results may not be meaningful. 
As the model is rather complex, we cancel the calculation of second derivatives. If you want to control the parameters, change the name of the algorithm in the TOML file from "automatic" to "simple_bounds" 
Optimization algorithm: hybrid Newton/BFGS with simple bounds [simple_bounds] 
** Optimization: BFGS with trust region for simple bounds 
Iter.      ASC_CAR_mu       ASC_SM_mu      ASC_SM_var          B_COST          B_TIME     Function    Relgrad   Radius      Rho      
    0               1               1               2              -1              -1      5.4e+03      0.032        1     0.43    + 


In [ ]:
pandas_results

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR_mu,0.394659,0.354601,1.112964,2.657240e-01
ASC_CAR_var,0.313032,0.318754,0.982049,3.260755e-01
ASC_SM_mu,1.019348,0.264409,3.855192,1.156390e-04
ASC_SM_var,3.730982,0.257232,14.504375,0.000000e+00
ASC_TRAIN_var,0.793294,0.208461,3.805486,1.415261e-04
B_COST,-1.865443,0.212572,-8.775582,0.000000e+00
B_TIME,-1.762190,0.244854,-7.196915,6.159517e-13


## time and cost correlated

In [23]:
B_COST = Beta('B_COST', 0.0, None, None, 0)
B_COST_S = Beta('B_COST_S', 1, None, None, 0)
cost_draws = bioDraws('b_cost_rnd', 'NORMAL_HALTON3')

B_TIME = Beta('B_TIME', 0.0, None, None, 0)
B_TIME_S = Beta('B_TIME_S', 1, None, None, 0)
time_draws = bioDraws('b_time_rnd', 'NORMAL_HALTON3')

B_COST_TIME_S = Beta('B_COST_TIME_S', 1, None, None, 0)

B_COST_RND = B_COST + B_COST_S * cost_draws + B_COST_TIME_S * time_draws
B_TIME_RND = B_TIME + B_TIME_S * time_draws #+  B_COST_TIME_S * cost_draws

ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 1)
ASC_SM = Beta('ASC_SM', 0, None, None, 0)

V1 = ASC_TRAIN + B_TIME_RND * TRAIN_TT_SCALED + B_COST_RND * TRAIN_COST_SCALED
V2 = ASC_SM + B_TIME_RND * SM_TT_SCALED + B_COST_RND * SM_COST_SCALED
V3 = ASC_CAR + B_TIME_RND * CAR_TT_SCALED + B_COST_RND * CAR_CO_SCALED

V = {1: V1, 2: V2, 3: V3}
av = {1: TRAIN_AV_SP, 2: SM_AV, 3: CAR_AV_SP}

prob = models.logit(V, av, CHOICE)
#logprob = log(MonteCarlo(PanelLikelihoodTrajectory(prob)))
logprob = log(MonteCarlo(prob))

the_biogeme = bio.BIOGEME(
    database, logprob, number_of_draws=1000, seed=777
)
the_biogeme.modelName = 'b05normal_test'
the_biogeme.generate_pickle = False
the_biogeme.generate_html = False

Biogeme parameters read from biogeme.toml. 


In [24]:
the_biogeme.calculate_init_likelihood()

-6865.358935948356

In [25]:
results = the_biogeme.estimate()
pandas_results = results.get_estimated_parameters()

print(results.short_summary())

As the model is rather complex, we cancel the calculation of second derivatives. If you want to control the parameters, change the name of the algorithm in the TOML file from "automatic" to "simple_bounds" 
*** Initial values of the parameters are obtained from the file __b05normal_test.iter 
Parameter values restored from __b05normal_test.iter 
The number of draws (1000) is low. The results may not be meaningful. 
As the model is rather complex, we cancel the calculation of second derivatives. If you want to control the parameters, change the name of the algorithm in the TOML file from "automatic" to "simple_bounds" 
Optimization algorithm: hybrid Newton/BFGS with simple bounds [simple_bounds] 
** Optimization: BFGS with trust region for simple bounds 
Iter.         ASC_CAR          ASC_SM          B_COST        B_COST_S   B_COST_TIME_S          B_TIME        B_TIME_S     Function    Relgrad   Radius      Rho      
    0            0.38            0.35              -2             1.6 

Results for model b05normal_test
Nbr of parameters:		7
Sample size:			6768
Excluded data:			3960
Final log likelihood:		-5162.165
Akaike Information Criterion:	10338.33
Bayesian Information Criterion:	10386.07



In [26]:
pandas_results

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,0.379300,0.056364,6.729448,1.703082e-11
ASC_SM,0.349603,0.065062,5.373407,7.726263e-08
B_COST,-1.943150,0.101968,-19.056481,0.000000e+00
B_COST_S,1.435057,0.087114,16.473322,0.000000e+00
B_COST_TIME_S,0.154390,0.087114,1.772277,7.634861e-02
B_TIME,-2.420522,0.125029,-19.359725,0.000000e+00
B_TIME_S,1.692977,0.146641,11.545012,0.000000e+00
